In [1]:
# Importamos las librerías necesarias

import sys
import os
from sqlalchemy import create_engine, text
sys.path.append(os.path.abspath("..")) # Para importar desde el directorio padre
from config.settings import DB_CONNECTION_STRING
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns


# Exploratory Data Analysis (EDA)

## Conexión base de datos y carga del Dataset

In [2]:
# Leer el archivo SQL
sql_path = os.path.join("..", "sql_queries", "queries.sql") # Ruta al archivo SQL
with open(sql_path, "r", encoding="utf-8") as file:
    query = file.read()

# Parámetros para la consulta
params = {
    "medico": "PSICOLOGÍA",
    "fechaini": "20230101",
    "fechafin": "20250504"
}

# Crear conexión y ejecutar
engine = create_engine(DB_CONNECTION_STRING)
with engine.connect() as conn:
    historias_clinicas = pd.read_sql(text(query), conn, params=params)

historias_clinicas = historias_clinicas.head(10000) # Limitar a 10,000 filas para pruebas iniciales


## Exploración Inicial 

In [3]:
# Dimensiones
dimensiones = historias_clinicas.shape
print(dimensiones)

(10000, 7)


In [4]:
# Tipos de datos
historias_clinicas.dtypes

SEXO                   object
EDAD                    int64
GRUPO                  object
ESPECIALIDAD_MEDICA    object
SUBJETIVO              object
OBJETIVO               object
Concatenada            object
dtype: object

In [5]:
historias_clinicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   SEXO                 10000 non-null  object
 1   EDAD                 10000 non-null  int64 
 2   GRUPO                10000 non-null  object
 3   ESPECIALIDAD_MEDICA  10000 non-null  object
 4   SUBJETIVO            9749 non-null   object
 5   OBJETIVO             9779 non-null   object
 6   Concatenada          10000 non-null  object
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [6]:
# Revisamos la distribución de la variable objetivo
historias_clinicas["GRUPO"].value_counts()

GRUPO
T. de Ansiedad        3223
T. Depresivos         2211
Otros Trastornos      1818
T. Personalidad       1260
T. Externalizantes     991
T. de Adaptación       497
Name: count, dtype: int64

**OBSERVACIONES: Se puede notar que la columna "GRUPO" presenta un desvanlanceo considerable en cada una de las respectivas clases.**

In [7]:
# Transformamos los nombres de las columnas a minúsculas manteniendo el snake case
columns_names = historias_clinicas.columns.str.lower() # Convertir a minúsculas
historias_clinicas.columns = columns_names

In [8]:
# Check column names
print(historias_clinicas.columns)

Index(['sexo', 'edad', 'grupo', 'especialidad_medica', 'subjetivo', 'objetivo',
       'concatenada'],
      dtype='object')


# Validación Registros Duplicados

In [9]:
# Duplicated rows
print(historias_clinicas.duplicated().sum())

216


In [10]:
# Filtramos filas duplicadas
historias_clinicas[historias_clinicas.duplicated()].head(10)

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada
70,Femenino,36,T. Externalizantes,PSICOLOGÍA,Paciente refiere que presenta baja tolerancia ...,"Paciente quien evalúo por primera vez, con bas...",Paciente refiere que presenta baja tolerancia ...
71,Femenino,28,T. Externalizantes,PSICOLOGÍA,Paciente refiere que actualmente tiene una rel...,"Paciente quien evalúo por primera vez, pacient...",Paciente refiere que actualmente tiene una rel...
86,Femenino,41,T. Externalizantes,PSICOLOGÍA,Paciente refiere que que actualemente se encue...,"Paciente quien evalúo por primera vez, alerta,...",Paciente refiere que que actualemente se encue...
95,Masculino,36,T. de Adaptación,PSICOLOGÍA,"El paciente refiere: ""Tuve que salir de mi hog...","Paciente de 35 años, no refiere antecedentes d...","El paciente refiere: ""Tuve que salir de mi hog..."
120,Femenino,42,T. Externalizantes,PSICOLOGÍA,Siente que no vivió el duelo del proceso de se...,"Paciente quien evalúo por primera vez, alerta,...",Siente que no vivió el duelo del proceso de se...
128,Femenino,27,T. Externalizantes,PSICOLOGÍA,Paciente refiere que ha presentado ataques de ...,"Paciente quien evalúo por primera vez, pacient...",Paciente refiere que ha presentado ataques de ...
129,Femenino,42,T. Externalizantes,PSICOLOGÍA,Siente que no vivió el duelo del proceso de se...,"Paciente quien evalúo por primera vez, alerta,...",Siente que no vivió el duelo del proceso de se...
130,Femenino,9,Otros Trastornos,PSICOLOGÍA,"La madre refiere ""Ella ha manifestado ciertas ...","Paciente ingresa en compañía de la madre, aler...","La madre refiere ""Ella ha manifestado ciertas ..."
193,Femenino,28,T. Externalizantes,PSICOLOGÍA,"Paciente refiere "" Hace un tiempo yo estaba en...","Paciente quien evalúo por primera vez, pacient...","Paciente refiere "" Hace un tiempo yo estaba en..."
241,Femenino,23,T. Externalizantes,PSICOLOGÍA,"Paciente refiere ""Yo era una persona muy imspu...","Paciente quien evalúo por primera vez, pacien...","Paciente refiere ""Yo era una persona muy imspu..."


**OBSERVACIONES: La unica columna con valores repetidos es la columna "especialidad_medica", no hay filas con registros diplicados en todas las columnas. Por lo tanto, no es necesario eliminar filas duplicadas o se perderia información importante del dataset.**

# Validación Valores Nulos 

In [11]:
# Nulos

print(historias_clinicas.isnull().sum())

sexo                     0
edad                     0
grupo                    0
especialidad_medica      0
subjetivo              251
objetivo               221
concatenada              0
dtype: int64


In [12]:
# Filtramos las filas con valores nulos
historias_clinicas[historias_clinicas.isnull().any(axis=1)].head(10) # Filas con valores nulos en cualquier columna

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada
0,Masculino,10,T. Externalizantes,PSICOLOGÍA,None,None,
3,Masculino,9,Otros Trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",None,"La madre refiere ""el viene por un acompañamien..."
56,Masculino,33,Otros Trastornos,PSICOLOGÍA,Paciente desde 16 años,None,Paciente desde 16 años
172,Masculino,17,T. Externalizantes,PSICOLOGÍA,"Yo antes sabia controlar ira, estoy teniendo m...",None,"Yo antes sabia controlar ira, estoy teniendo m..."
250,Masculino,17,T. Externalizantes,PSICOLOGÍA,"Yo antes sabia controlar ira, estoy teniendo m...",None,"Yo antes sabia controlar ira, estoy teniendo m..."
266,Masculino,14,T. de Adaptación,PSICOLOGÍA,None,"La mamá del paciente se percibe alerta, normoc...","La mamá del paciente se percibe alerta, normo..."
273,Masculino,34,T. Externalizantes,PSICOLOGÍA,None,El paciente presenta una apariencia general ap...,El paciente presenta una apariencia general a...
277,Femenino,33,Otros Trastornos,PSICOLOGÍA,None,La paciente presenta una apariencia general ap...,La paciente presenta una apariencia general a...
284,Masculino,14,T. de Adaptación,PSICOLOGÍA,None,El paciente presenta una apariencia general ap...,El paciente presenta una apariencia general a...
288,Masculino,19,T. de Ansiedad,PSICOLOGÍA,None,El paciente presenta una apariencia general ap...,El paciente presenta una apariencia general a...


**OBSERVACIÓN: Despues de filtrar las columnas con valores nulos, se pudo notar que para los casos donde las columnas "subjetivo" y "objetivo" tienen valores nulos la columna "concatenada" no los tenia por lo tanto se procede a eliminar los registros donde ambas columnas tengan nulos al mismo tiempo para garantizar que la columna "concatenada" no tenga string vacios o algún tipo de de dato diferente.**

In [13]:
# Eliminamos las filas con valores nulos en ambas columnas "subjetivo" y "objetivo"

historias_clinicas = historias_clinicas.dropna(subset=["subjetivo", "objetivo"], how="all") # Eliminar filas donde ambas columnas son nulas 

In [14]:
historias_clinicas.shape

(9809, 7)

In [15]:
# Validación Valores Nulos

print(historias_clinicas.isnull().sum())

sexo                    0
edad                    0
grupo                   0
especialidad_medica     0
subjetivo              60
objetivo               30
concatenada             0
dtype: int64


In [16]:
# Filtramos las filas con valores nulos nuevamente para verificar

historias_clinicas[historias_clinicas.isnull().any(axis=1)].head(10) # Filas con valores nulos en cualquier columna

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada
3,Masculino,9,Otros Trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",None,"La madre refiere ""el viene por un acompañamien..."
56,Masculino,33,Otros Trastornos,PSICOLOGÍA,Paciente desde 16 años,None,Paciente desde 16 años
172,Masculino,17,T. Externalizantes,PSICOLOGÍA,"Yo antes sabia controlar ira, estoy teniendo m...",None,"Yo antes sabia controlar ira, estoy teniendo m..."
250,Masculino,17,T. Externalizantes,PSICOLOGÍA,"Yo antes sabia controlar ira, estoy teniendo m...",None,"Yo antes sabia controlar ira, estoy teniendo m..."
266,Masculino,14,T. de Adaptación,PSICOLOGÍA,None,"La mamá del paciente se percibe alerta, normoc...","La mamá del paciente se percibe alerta, normo..."
273,Masculino,34,T. Externalizantes,PSICOLOGÍA,None,El paciente presenta una apariencia general ap...,El paciente presenta una apariencia general a...
277,Femenino,33,Otros Trastornos,PSICOLOGÍA,None,La paciente presenta una apariencia general ap...,La paciente presenta una apariencia general a...
284,Masculino,14,T. de Adaptación,PSICOLOGÍA,None,El paciente presenta una apariencia general ap...,El paciente presenta una apariencia general a...
288,Masculino,19,T. de Ansiedad,PSICOLOGÍA,None,El paciente presenta una apariencia general ap...,El paciente presenta una apariencia general a...
289,Femenino,11,T. de Adaptación,PSICOLOGÍA,None,La paciente y su madre presentan una aparienci...,La paciente y su madre presentan una aparienc...


## Resumen de Estadisticas

In [17]:
historias_clinicas.describe(include="all").T # Transponemos la tabla para mejor visualización

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
sexo,9809,2,Femenino,6763,NaN,NaN,NaN,NaN,NaN,NaN,NaN
edad,9809.0,NaN,NaN,NaN,32.793659,18.820209,1.0,17.0,29.0,46.0,94.0
grupo,9809,6,T. de Ansiedad,3179,NaN,NaN,NaN,NaN,NaN,NaN,NaN
especialidad_medica,9809,1,PSICOLOGÍA,9809,NaN,NaN,NaN,NaN,NaN,NaN,NaN
subjetivo,9749,9549,,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
objetivo,9779,6184,"Paciente quien evalúo por teleconsulta, se evi...",221,NaN,NaN,NaN,NaN,NaN,NaN,NaN
concatenada,9809,9643,La paciente presenta una apariencia general a...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Poner la primera letra de cada palabra en mayúscula en la columna "grupo"
historias_clinicas["grupo"] = historias_clinicas["grupo"].str.capitalize()

In [19]:
historias_clinicas

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada
1,Femenino,38,Otros trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...","Paciente refiere: ""Me empezaron a dar como uno..."
2,Masculino,22,T. externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...","Paciente refiere ""Me he sentido muy mal, en el..."
3,Masculino,9,Otros trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",None,"La madre refiere ""el viene por un acompañamien..."
4,Masculino,28,Otros trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...","Paciente refiere ""Estas cosas que han pasado m..."
5,Femenino,8,Otros trastornos,PSICOLOGÍA,"La madre refiere ""Ella ha manifestado ciertas ...","Paciente ingresa en compañía de la madre, aler...","La madre refiere ""Ella ha manifestado ciertas ..."
...,...,...,...,...,...,...,...
9995,Femenino,60,Otros trastornos,PSICOLOGÍA,"Paciente refiere: ""A mi me mandó el internista...","Paciente alerta, colaboradora con apariencia o...","Paciente refiere: ""A mi me mandó el internista..."
9996,Masculino,7,T. personalidad,PSICOLOGÍA,"Madre refiere: ""La verdad el comportamiento es...","Paciente alerta, colaboradora con apariencia o...","Madre refiere: ""La verdad el comportamiento es..."
9997,Femenino,34,T. de ansiedad,PSICOLOGÍA,"Refiere paciente:\n""yo me pongo como agresiva,...","Paciente alerta, colaboradora con apariencia o...","Refiere paciente:\n""yo me pongo como agresiva,..."
9998,Femenino,16,T. de ansiedad,PSICOLOGÍA,"La paciente refiere: ""Pues yo si quiero hacer ...","Paciente alerta, colaboradora con apariencia o...","La paciente refiere: ""Pues yo si quiero hacer ..."


In [26]:
# Funcion guardar las imagenes

# Guardar imagen del boxplot en una carpeta llamada "imagenes_eda"
dir_imagenes = "imagenes_eda"
if not os.path.exists(dir_imagenes):
    os.makedirs(dir_imagenes)

# Funcion para guardar las imagenes

# Función para guardar imágenes generadas por Plotly en la carpeta "imagenes_eda"
def save_plotly_fig(fig, fig_name, fig_extension="png", scale=2):
    path = os.path.join(dir_imagenes, f"{fig_name}.{fig_extension}")
    fig.write_image(path, format=fig_extension, scale=scale)

In [31]:
# Boxplot columna edad con Plotly
fig = px.box(
    historias_clinicas,
    y="edad",
    title="Boxplot de la columna Edad",
    color_discrete_sequence=["#FA7C63"]  # Color personalizado
)

fig.update_layout(title_x=0.5)  # Centra el título

fig.show()

# Guardar la imagen
save_plotly_fig(fig, "boxplot_edad")

## Visualiazción de Datos: Categoricos

In [24]:
# Función para guardar imágenes generadas por Plotly en la carpeta "imagenes_eda"
def save_plotly_fig(fig, fig_name, fig_extension="png", scale=2):
    path = os.path.join(dir_imagenes, f"{fig_name}.{fig_extension}")
    fig.write_image(path, format=fig_extension, scale=scale)


In [23]:
# Distribución de la columna sexo

fig = px.histogram(
    historias_clinicas,
    x="sexo",
    title="Distribución de la columna Sexo",
    color="sexo",
    text_auto=True
)

fig.update_layout(
    xaxis_title="Sexo del Paciente",
    yaxis_title="Cantidad de Registros",
    title_x=0.5  # Centra el título
)

fig.show()

# guardar la imagen 
save_plotly_fig(fig, "distribucion_sexo")


In [32]:
# Distribucion de la columna grupo
fig = px.histogram(
    historias_clinicas,
    x="grupo",
    title="Distribución de la columna Grupo",
    color="grupo",
    text_auto=True
)

fig.update_layout(
    xaxis_title="Grupo del Paciente",
    yaxis_title="Cantidad de Registros",
    title_x=0.5  # Centra el título
)

fig.show()

# guardar la imagen
save_plotly_fig(fig, "distribucion_grupo")

## Visualización de Datos: Valores Numericos

In [33]:
# Histograma de edad
fig = px.histogram(historias_clinicas, 
             x="edad", 
             title="Distribución de Edad en Historias Clínicas",
             marginal="box", # Añade un boxplot en la parte superior
            text_auto=True)

fig.update_layout(
    xaxis_title="Edad del Paciente",
    yaxis_title="Cantidad de Registros",
    title_x=0.5  # Centra el título
)

fig.update_layout(width=2500, height=500)

fig.show()

# guardar la imagen
save_plotly_fig(fig, "histograma_edad")

In [34]:
# Distribución por edad y sexo
fig = px.histogram(historias_clinicas, 
             x="edad", 
             title="Distribución de edad en Historias Clínicas por sexo", 
             color="sexo",
             text_auto=True
             )


fig.update_layout(
    xaxis_title="Edad del Paciente",
    yaxis_title="Cantidad de Registros",
    title_x=0.5  # Centra el título
)


fig.update_layout(width=2000, height=500)


fig.show()

# guardar la imagen
save_plotly_fig(fig, "histograma_edad_sexo")

In [35]:
# Distribución por grupo y sexo
fig = px.histogram(historias_clinicas, 
             x="grupo", 
             title="Distribución de Grupo en Historias Clínicas por Sexo", 
             color="sexo",
             text_auto=True
             )

fig.update_layout(
    xaxis_title="Grupo del Paciente",
    yaxis_title="Cantidad de Registros",
    title_x=0.5  # Centra el título
)

fig.update_layout(width=2000, height=500)

fig.show()

# guardar la imagen
save_plotly_fig(fig, "distribucion_grupo_sexo")
